In [1]:
import os
import re
import pandas as pd
import seaborn as sns
from bs4 import BeautifulSoup

In [5]:
folder = os.path.join(os.getcwd(),'..','data','korpus')
subjects = ['Academic','Culture','European','Law','News',
            'Opinion','Parliament','Religion','Sport']

## Removing XML tags from <i>korpus</i>

All tags except <b>\<s\></b> will be removed from the korpus.

In [ ]:
regex = re.compile(r'<(?![/]?s).*?>')
def clean(text):
    return regex.sub('', text)

In [ ]:
for sbj in subjects:
    path = os.path.join(folder,sbj)
    
    for filename in os.listdir(path):
        filepath = os.path.join(path,filename)

        with open(filepath, 'r', encoding='utf-8') as f:
            print(filename)
            text = f.read()
        
        with open(filepath, 'w', encoding='utf-8') as f:
            f.write(clean(text))
    
    

## Turning text file into dataset

In [3]:
subjects = ['News']

for sbj in subjects:
    data = []
    path = os.path.join(folder,sbj)
    
    for filename in os.listdir(path):
        filepath = os.path.join(path,filename)
        
        with open(filepath,'r', encoding='utf-8') as f:
            print(filename)
            lines = f.readlines()
            
            for l in lines:
                d = l.split('\t')
                
                #Start and End Token
                if len(d) == 1:
                    if re.search(r'<s id="[0-9]*">', d[0]): data.append(['<s>','START',None,None])
                    elif re.search(r'</s>', d[0]):          data.append(['</s>','END',None,None])
                
                elif len(d) > 1:
                    d[-1] = d[-1][:-1]
                    data.append(d)
                    
    df = pd.DataFrame(data, columns=['Word','POS','Lemma','Root'])
    df.to_csv(os.path.join(path,sbj+'.csv'), index=False)

News2.txt


## Joining individual csv files

We will create 2 versions of korpus.csv. One will simply be all the csv files together while the other will read an equal number of bytes from each file. This is so the frequency counts won't be biased based on the subject of the texts.

In [6]:
with open(os.path.join(folder,'korpus.csv'), 'w', encoding='utf-8') as f1:
    for sbj in subjects:
        with open(os.path.join(folder,sbj,sbj+'.csv'), 'r', encoding='utf-8') as f2:
            f1.write('\n')
            f1.write(f2.read())
            print(f'{sbj} Finished')

print('All Finished\n')


min_size = min([os.path.getsize(os.path.join(folder,sbj,sbj+'.csv')) for sbj in subjects])

#Only read min_length lines from each csv file. 

with open(os.path.join(folder,'norm_korpus.csv'), 'w', encoding='utf-8') as f1:
    for sbj in subjects:
        with open(os.path.join(folder,sbj,sbj+'.csv'), 'r', encoding='utf-8') as f2:    
            f1.write('\n')
            f1.write(f2.read(min_size))
            print(f'{sbj} Finished')

print('All Finished')


Academic Finished
Culture Finished
European Finished
Law Finished
News Finished
Opinion Finished
Parliament Finished
Religion Finished
Sport Finished
All Finished

Academic Finished
Culture Finished
European Finished
Law Finished
News Finished
Opinion Finished
Parliament Finished
Religion Finished
Sport Finished
All Finished


## Exploring Dataset

In [94]:
%%time

#20 million rows
df = pd.read_csv(os.path.join(folder,'korpus.csv'),
                 usecols=["Word","POS"],
                 dtype={"Word": "U","POS": "S"},
                 nrows = 20000000)

df_norm = pd.read_csv(os.path.join(folder,'norm_korpus.csv'),
                      usecols=["Word","POS"],
                      dtype={"Word": "U","POS": "S"})

df.head(10)

Wall time: 7.89 s


,Word,POS
0,<s>,START
1,L-,DEF
2,għan,NOUN
3,prinċipali,ADJ
4,ta',GEN
5,Conectando,NOUN-PROP
6,Mundos,NOUN-PROP
7,(,X-PUN
8,Malta,NOUN-PROP
9,),X-PUN


### Cleaning

In [95]:
df['POS'].unique()

array(['START', 'DEF', 'NOUN', 'ADJ', 'GEN', 'NOUN-PROP', 'X-PUN',
       'X-ABV', 'PREP', 'CONJ-CORD', 'PART-PASS', 'PREP-DEF', 'PRON-PERS',
       'COMP', 'VERB', 'END', 'LIL-DEF', 'CONJ-SUB', 'KIEN', 'GEN-PRON',
       'ADV', 'VERB-PSEU', 'NEG', 'GEN-DEF', 'QUAN', 'PRON-DEM', 'X-DIG',
       'PRON-INT', 'FOC', 'PREP-PRON', 'NUM-WHD', 'LIL', 'NUM-CRD',
       'X-ENG', 'X-FOR', 'PROG', 'INT', 'X-BOR', 'PRON-PERS-NEG',
       'LIL-PRON', 'PRON-INDEF', 'PRON-DEM-DEF', 'NUM-ORD', 'HEMM',
       'PRON-REF', 'PART-ACT', 'FUT', 'NUM-FRC', 'PRON-REC', 'POS'],
      dtype=object)

In [96]:
%%time
#Maltese Tagset: https://mlrs.research.um.edu.mt/resources/malti03/tagset30.html

df = df.drop(df[df['POS']=='X-PUN'].index) #Punctuation
# df = df.drop(df[df['POS']=='X-DIG'].index) #Digits
# df = df.drop(df[df['POS']=='X-ENG'].index) #English
# df = df.drop(df[df['POS']=='X-FOR'].index) #Foreign
# df = df.drop(df[df['POS']=='X-ABV'].index) #Abbreviations
df = df.drop(df[df['POS']=='X-BOR'].index) #Gibberish
df = df.drop(df[df['POS']=='INT'].index)   #Interjections


df_norm = df_norm.drop(df_norm[df_norm['POS']=='X-PUN'].index) #Punctuation
# df_norm = df_norm.drop(df_norm[df_norm['POS']=='X-DIG'].index) #Digits
# df_norm = df_norm.drop(df_norm[df_norm['POS']=='X-ENG'].index) #English
# df_norm = df_norm.drop(df_norm[df_norm['POS']=='X-FOR'].index) #Foreign
# df_norm = df_norm.drop(df_norm[df_norm['POS']=='X-ABV'].index) #Abbreviations
df_norm = df_norm.drop(df_norm[df_norm['POS']=='X-BOR'].index) #Gibberish
df_norm = df_norm.drop(df_norm[df_norm['POS']=='INT'].index)   #Interjections


Wall time: 9.24 s


In [97]:
percent_missing = df.isnull().sum() * 100 / len(df)
missing_value_df = pd.DataFrame({'column_name': df.columns,
                                 'percent_missing': percent_missing})

missing_value_df

,column_name,percent_missing
Word,Word,0.000382
POS,POS,0.000000


In [98]:
df = df.dropna(subset=['Word'])
df = df.drop(df[df["Word"]=='"'].index)
df = df.drop(df[df["Word"]=='&lt'].index)
df = df.drop(df[df["Word"]=='&gt'].index)
df = df.drop(df[df["Word"]=='&amp'].index)

df_norm = df_norm.dropna(subset=['Word'])
df_norm = df_norm.drop(df_norm[df_norm["Word"]=='"'].index)
df_norm = df_norm.drop(df_norm[df_norm["Word"]=='&lt'].index)
df_norm = df_norm.drop(df_norm[df_norm["Word"]=='&gt'].index)
df_norm = df_norm.drop(df_norm[df_norm["Word"]=='&amp'].index)

In [99]:
percent_missing = df.isnull().sum() * 100 / len(df)
missing_value_df = pd.DataFrame({'column_name': df.columns,
                                 'percent_missing': percent_missing})

missing_value_df

,column_name,percent_missing
Word,Word,0.0
POS,POS,0.0


In [100]:
%%time
df.to_csv(os.path.join(folder,'korpus_clean.csv'), index=False)
df_norm.to_csv(os.path.join(folder,'norm_korpus_clean.csv'), index=False)

Wall time: 32.8 s


## Frequency Counts

In [110]:
df = pd.read_csv(os.path.join(folder, "korpus_clean.csv"),
                 usecols=["Word","POS"],
                 dtype={"Word": "U","POS": "S"})

df_norm = pd.read_csv(os.path.join(folder, "norm_korpus_clean.csv"),
                 usecols=["Word","POS"],
                 dtype={"Word": "U","POS": "S"})


In [119]:
#Get all unique words
df_frequency = df.value_counts().to_frame()[0].reset_index()
df_frequency.columns = ['Word','POS','Frequency']


df_normal_frequency = df_norm.value_counts().to_frame()[0].reset_index()
df_normal_frequency.columns = ['Word','POS','Frequency']

df_frequency.to_csv(os.path.join(folder,'korpus_frequency.csv'), index=False)
df_normal_frequency.to_csv(os.path.join(folder,'norm_korpus_frequency.csv'), index=False)

df_frequency

,Word,POS,Frequency
0,l-,DEF,716515
1,<s>,START,713350
2,</s>,END,713349
3,ta',GEN,702736
4,li,COMP,463290
...,...,...,...
266540,Norm,NOUN-PROP,1
266541,jeħduni,VERB,1
266542,jeħdulhom,VERB,1
266543,Normalemnt,ADV,1


### Viewing most common nouns, adjectives or verbs.

In [108]:
df_frequency[(df_frequency["POS"] == "NOUN")|
             (df_frequency["POS"] == "ADJ") |
             (df_frequency["POS"] == "VERB")]

,Word,POS,Frequency
26,Regolament,NOUN,63056
27,Artikolu,NOUN,61768
32,Kummissjoni,NOUN,55794
39,Membri,NOUN,47351
41,Direttiva,NOUN,46369
...,...,...,...
266536,jeħhilsu,VERB,1
266537,jeħfiefu,VERB,1
266538,Norges,VERB,1
266541,jeħduni,VERB,1


In [109]:
df_normal_frequency[(df_normal_frequency["POS"] == "NOUN")|
                    (df_normal_frequency["POS"] == "ADJ") |
                    (df_normal_frequency["POS"] == "VERB")]

,Word,POS,Frequency
35,ta',VERB,9188
41,sena,NOUN,8152
46,oħra,ADJ,7593
81,jista',VERB,5097
85,Ministru,NOUN,4797
...,...,...,...
142092,Mewġiet,NOUN,1
142094,Mexxa,NOUN,1
142095,jippanjaw,VERB,1
142096,jippanikjax,VERB,1
